In [3]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re
import time
from datetime import date
import traceback
import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from geocube.api.core import make_geocube
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys

# Tess' Goals

I am interested in tying lighting strike data to individuals fires to contrsuct a timeline of when fires in 2023 Canada could have ignited compared to when satellites detected them. To do this, I have access to a high resolution dataset of global lightning strikes down to the millisecond. The data is stored as daily `.raw` files, each with global coverage and ~200000 rows. A public copy of the data is stored at `s3://maap-ops-workspace/shared/tmccabe/Lighting_data` (See Notes). I have been trying to find effecient ways to construct timeseries for specific regions out of these files. Ideally I would like to be able to write out a non-kernel-crashing file that is "Canada-2032-Jan-Oct.csv" or similar. 


Note:

- Because I ran into "Transport endpoint not conected" errors at times during debugging, I also have a private copy of the data on /projects. 
- Based on advice, I also transformed each file into a `.parquet` file, also stored on /projects. 

# Things Tess has Tried: For loop with frequent write-outs

I tried writing a for loop that writes-out every 5 files. The Kernel crashes after ~5 files, certain files too big for memory at all. 

In [ ]:
##### "For Loop Style Read-in" #####


def get_ca_lt(date, bbox = [-83.69877641421793, 44.25483911637959, -55, 62.94135765648493]): #[-169, 44, -48, 75]
    CA_bbox = box(bbox[0], bbox[1], bbox[2], bbox[3])
    print(date)
    
    #lt = pd.read_csv("/projects/my-public-bucket/Lighting_data/2023/englnrt_"+ date+"_daily_v1_lit.raw", names =["InterCloud",
    lt = pd.read_csv("/projects/2023_lightning_data/englnrt_"+ date+"_daily_v1_lit.raw", names =["InterCloud",
                                                                                             "t", 
                                                                                             "lat", 
                                                                                             "lon",
                                                                                             "current_mag", 
                                                                                             "multiplicity_0", 
                                                                                             "accr", 
                                                                                             "error_elps", 
                                                                                             "num_station"])
    lt =  gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt['lon'], lt['lat']))
    lt.t = lt.t.astype("datetime64[ns]")
    lt = lt[lt.InterCloud == 0]
    lt = lt.set_crs("epsg:4326")
    smol = lt.clip(CA_bbox)
    return(smol)

def lt_timeseries_extract(date_snap, region, sub_region, run_name): ## putting into function because might help memory???
    
    run_name = str(date.today()) + run_name_st
    lt = [] # full info per day
    bad_ids = [] # list of IDs that failed due to error
    bbox = [-83.69877641421793, 44.25483911637959, -55, 62.94135765648493]
    for n,i in enumerate(date_snap, start = 0):

        try:
            smol = get_ca_lt(str(i), bbox = bbox )

        except Exception as e:
            print("Error at date: ",n,)
            bad_log = traceback.format_exc(limit = 40) # Get log of error
            print(bad_log)
            bad_ids.append(
            {
                "date": n, 
                "log": bad_log
            }
            )
        lt.append(smol)
        if((n%5 == 0) | (n == (len(date_range) -1))):
            print("")
            manyfr = gpd.GeoDataFrame(pd.concat(lt, ignore_index=True), crs= smol.crs)
            manyfr.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/Lt_" + region + "_" + sub_region +  "_" + run_name +".csv")
            debug = pd.DataFrame(bad_ids)
            debug.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/DEBUG_" + region + "_" + sub_region + "_" + run_name + ".csv")
        
    return(manyfr)

date_range = pd.date_range(start = "2023-05-01 00:00:00", end = "2023-07-20 00:00:00", freq="24H") 
date_snap = date_range.strftime("%Y%m%d")


start = time.time()

################# Horrible for loop ##################
sub_region = "example_Quebec"
region =  "example_CA"
run_name_st = "example"
manyfr = lt_timeseries_extract(date_snap = date_snap, region = region,  sub_region = sub_region, run_name = run_name_st) ## putting into function because might help memory???

# Grab Currrent Time After Running the Code
end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time)
      

In [ ]:
### When for loop kills Kernel and dies, I can read in all the partial write-outs

def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/Lt_CA_Quebec_*.csv")

manyfr= concat_subsets(files)

This has been my most successfull technique because it gives me output I can plot. It's horrible becuase it means I need to supervise it and wait for it to die, restart it with a new name, and wait again. It also doesn't work at all for some files. 

# Things Tess has Tried: Multiprocess 

I've tried a python script using multiprocessing based on an example a colleague (Jordan Caraballo-Vega - Thanks for the help!) provided. It executed, but when I watched it with `top` I noticed that all the parallel sub-tests would be get allocated, and take up CPU and memory, but then slowly all the tasks would be paused until just one was running, and I assume ran until it got killed. Nothing was output. Here is the `.py` file: 

I moved to dask hoping to use it to diagnose what the computational limit multiprocessing was hitting was. 


In [ ]:
import sys
import pandas as pd
import geopandas as gpd
from glob import glob
import time
from multiprocessing import Pool, cpu_count


csvs_regex = '/projects/2023_lightning_data/*.raw'

def read_gdf(filename):
    af_df = pd.read_csv(filename, names =["InterCloud",
                                                                                             "t", 
                                                                                             "lat", 
                                                                                             "lon",
                                                                                             "current_mag", 
                                                                                             "multiplicity_0", 
                                                                                             "accr", 
                                                                                             "error_elps", 
                                                                                             "num_station"])
    # convert to GeoDataFrame
    af_gdf = gpd.GeoDataFrame(
        af_df,
        geometry=gpd.points_from_xy(af_df.lon, af_df.lat),
        crs="EPSG:4326"
    )
    print(filename)
    return af_gdf


def main():

    csvs_list = glob(csvs_regex)
    print(len(csvs_list))
    start = time.time()
    print(start)

    with Pool(processes=(cpu_count()) - 20) as p:
        result = p.map(read_gdf, csvs_list)
    gdf = pd.concat(result)
    print(gdf.shape)
    gdf.to_file("/projects/old_shared/fire_weather_vis/Lightning_analysis/test_para/2023.gpkg") # Nothing ever wrote to here
    end = time.time()

    total_time = end - start
    print("\n"+ str(total_time))
    
if __name__ == "__main__":
    sys.exit(main())
    

# Things Tess has Tried: Dask-delaying my exisitng functions

I first tried to use dask delay on the functions I wrote for my for-loop. Once I figured out not to instantiate the client through the tab (Thanks Alex & Julia), I got garbage collection warnings for a while, until either the kernel crashed, or I lost patience. Again, nothing was written out. 

In [ ]:
import dask
from dask.distributed import Client, progress

client = Client(threads_per_worker=2, n_workers=3) # I have also tried with 1 worker. Same results AFAIK. 
client

lazy_results = []

for d in date_snap:
    tmp = dask.delayed(get_ca_lt)(str(d))
    lazy_results.append(tmp)

futes = dask.persist(*lazy_results) 


results = dask.compute(*futes)

```
2023-10-05 07:16:28,594 - distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
```

# Things Tess has Tried: dask.dataframe and dask_geopandas

Thanks to anouther helpful suggestion + example by Jordan. I switched to using dask.dataframe and dask_geopandas. This seemed pretty close to working. It front-loaded the subsetting, let me read-in all my files at once, and even returned numbers for `ddf.head()` sometimes. However, sometimes when I did an intermediate `ddf.head()` or when I tried to plot the `day_strike` object, I just got garbage collection warnings. 

In [ ]:
import sys
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask_geopandas
from glob import glob
from shapely.geometry import box
import dask



from dask.distributed import Client, progress
client = Client(threads_per_worker=2, n_workers=3)
client

def lt_file_subset(csvs_regex = '/projects/2023_lightning_data/e*.raw'):
    
    bbox = [-83.69877641421793, 44.25483911637959, -55, 62.94135765648493]
    CA_bbox = box(bbox[0], bbox[1], bbox[2], bbox[3])
    
    ddf = dd.read_csv(csvs_regex, names =["InterCloud",
    "t", 
    "lat", 
    "lon",
    "current_mag", 
    "multiplicity_0", 
    "accr", 
    "error_elps", 
    "num_station"], 
                      dtype={
        'InterCloud': 'int', 
        'lat': 'str',
        'lon': 'str', 
        'current_mag': 'float64',
        'multiplicity_0': 'str', 
        'accr': 'float64',
        'error_elps': 'float64',
        'num_station': 'int'
    })
    ddf = ddf.set_geometry(
            dask_geopandas.points_from_xy(ddf, 'lon', 'lat')
    )

    ddf = ddf.clip(CA_bbox)
    ddf = ddf[ddf.InterCloud == 0]
    ddf.t = ddf.t.astype("datetime64[ns]")
    ddf["t_agg"] = ddf.t.dt.strftime("%Y-%m-%d %H:00:00")
    day_strike = ddf.groupby('t_agg').count() # Group into hourly resolution

    return(day_strike)

day_strike = lt_file_subset(csvs_regex = '/projects/2023_lightning_data/e*.raw')

plt.plot(day_strike.index.astype("datetime64[ns]"), day_strike.InterCloud) ## just garbage collection for a while